In [1]:
import json
import os
import pathlib

import pandas as pd

In [2]:
CURRENT_PATH = pathlib.Path().absolute()
PATH_TO_DATA = os.path.join(CURRENT_PATH, "..", "..", "..", "Data")

In [3]:
def get_dataset_artifacts_for_type(dataset: str, artifact_type: str, hasExtension=False):
    path_to_artifacts = get_path_to_artifacts(
        dataset, artifact_type, hasExtension=hasExtension)
    data = None
    with open(path_to_artifacts) as json_file:
        data = json.load(json_file)
    return data

In [4]:
def get_dataset_path(dataset_name: str):
    dataset_query = list(
        filter(lambda f: f[0] != ".", os.listdir(PATH_TO_DATA)))
    assert dataset_name in dataset_query, 'Could not find dataset: %s' % dataset_name
    return os.path.join(PATH_TO_DATA, dataset_name)


def get_path_to_artifacts(dataset: str, artifact_type: str, hasExtension=False):
    path_to_dataset = get_dataset_path(dataset)
    artifact_type_query = list(
        filter(lambda f: f[0] != ".", os.listdir(path_to_dataset)))
    artifact_type_file_name = artifact_type + "" if hasExtension else artifact_type + ".json"
    assert artifact_type_file_name in artifact_type_query, "Could not find artifact set: %s" % artifact_type_file_name
    return os.path.join(path_to_dataset, artifact_type_file_name)


def get_path_to_trace_matrices(dataset: str):
    path_to_dataset = get_dataset_path(dataset)
    assert "TraceMatrices" in os.listdir(
        path_to_dataset), "Cannot find trace matrices in %s" % dataset
    return os.path.join(path_to_dataset, "TraceMatrices")

In [5]:
linked_matrices = {
    "Requirements": [("Level_1_to_Level_2.csv", 0), ("Level_1_to_Level_3.csv", 0)],
    "Designs": [("Level_1_to_Level_2.csv", 1), ("Level_2_to_Level_3.csv", 0)],
    "Classes": [("Level_1_to_Level_3.csv", 1), ("Level_2_to_Level_3.csv", 1)]
}
source_matrices = {
    "Level_1_to_Level_2.csv": {
        0: "Requirements",
        1: "Designs"
    },
    "Level_1_to_Level_3.csv": {
        0: "Requirements",
        1: "Classes"
    },
    "Level_2_to_Level_3.csv": {
        0: "Designs",
        1: "Classes"
    }
}

In [7]:
def get_traced_artifacts(dataset: str, source_artifact_type: str, source_artifact_id: str):
    global_traced_artifacts = []
    path_to_trace_matrices = get_path_to_trace_matrices(dataset)
    linked_trace_matrices = linked_matrices[source_artifact_type]
    
    for trace_matrix_name, target_axis in linked_trace_matrices:
        # 1. Load paths
        source_axis = 1 - target_axis # 1 - 0 = 1 | 1 - 1 = 0 (flips bit)
        source_artifact_type = source_matrices[trace_matrix_name][source_axis]
        path_to_linked_matrix = os.path.join(path_to_trace_matrices, trace_matrix_name)
        
        # 2. Find Traced Artifacts
        trace_matrices = pd.read_csv(path_to_linked_matrix).set_index("id")
        artifacts_in_type = get_dataset_artifacts_for_type(dataset, source_artifact_type)["artifacts"]
        
        # 3. Get traced artifacts ids
        query = trace_matrices.loc[source_artifact_id] if target_axis == 0 else trace_matrices[source_artifact_id]
        traced_artifact_ids = query[query == 1].index
        
        # 4. Load Traced Artifacts
        for traced_artifact_id in traced_artifact_ids:
            traced_artifacts = list(filter(lambda artifact: artifact["id"] == traced_artifact_id, artifacts_in_type))
            global_traced_artifacts = global_traced_artifacts + traced_artifacts
            
    return global_traced_artifacts

In [8]:
t_artifacts = get_traced_artifacts("Drone", "Requirements", "RE-8")

In [9]:
t_artifacts

[{'id': 'DD-30',
  'body': 'Transition from ON_GROUND to AWAITING_TAKEOFF_CLEARANCE When a flight plan is assigned to a UAV in the {{ON_GROUND}} state it transitions to the {{AWAITING_TAKEOFF_CLEARANCE}} state.',
  'type': 'Designs'},
 {'id': 'DD-32',
  'body': 'Transition from TAKING_OFF to FLYING When a UAV in the TAKING_OFF state achieves the target altitude it transitions to FLYING state.',
  'type': 'Designs'},
 {'id': 'DD-26',
  'body': 'Activation state is ON_GROUND When a UAV is initially activated on the ground it is in the {{ON_GROUND}} state',
  'type': 'Designs'},
 {'id': 'DD-579',
  'body': 'Transition from IN_AIR to LANDING When a UAV is in {{IN_AIR}} state and receives a command to land it transitions to {{LANDING}} state',
  'type': 'Designs'},
 {'id': 'ManagedDrone.java',
  'body': 'package edu.nd.dronology.core.vehicle;\n\nimport java.util.*;\nimport java.util.concurrent.ArrayBlockingQueue;\nimport java.util.concurrent.CountDownLatch;\nimport java.util.concurrent.Exec